In [ ]:
# install metpy if needed
!pip install metpy

In [2]:
import scipy
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from utils import lw_clr
from metpy.units import units
import metpy.calc as mpcalc

In [3]:
# Load template forcing file with correct attributes
template = xr.open_dataset('copy_prr_2010-2014.nc')

# Load netcdf with era5 data for Buckinghorse for 2016
ds = xr.load_dataset('./era5_WY2016.nc')

In [5]:
# define target lat/lon for Buckinghorse SNOTEL
target_lat = 47.43
target_lon = -123.27

# select data for this grid cell and convert to dataframe
buck = ds.sel(latitude=target_lat, longitude=target_lon, method='nearest').to_dataframe()

# convert precip to kg m-2 s-1
buck['pptrate'] = buck['tp']/3.6

# compute wind speed from u and v components
buck['windspd'] = (buck['u10']**2 + buck['v10']**2)**0.5

# convert incoming SW Rad to W m-2
buck['SWRadAtm'] = buck['ssrd']/3600
# set all erroneous negative SWRadAtm values to 0
buck['SWRadAtm'] = buck['SWRadAtm'].apply(lambda x: max(0, x))

# convert incoming LW Rad to W m-2
buck['LWRadAtm'] = buck['strd']/3600

# rename temperature
buck['airtemp'] = buck['t2m']

# rename air pressure
buck['airpres'] = buck['sp']

# set dewpoint to airtemp for erroneous values where d2m>t2m
mask = buck['airtemp'] - buck['d2m'] < 0
buck.loc[mask, 'd2m'] = buck.loc[mask, 'airtemp']
# compute specific humidity from dewpoint
buck['spechum'] = mpcalc.specific_humidity_from_dewpoint(np.array(buck['sp']) * units['Pa'], 
                                                             np.array(buck['d2m'])*units['kelvin']).to('kg/kg')

# drop unnecessary columns
buck.drop(columns=['longitude', 'latitude', 'u10', 'v10', 'd2m', 't2m', 'sp', 'ssrd', 'strd', 'tp'], inplace=True)

In [ ]:
# create np arrays with metpy units
temperature=np.array(buck.airtemp)*units.kelvin
pressure = np.array(buck.airpres)*units.Pa
mr = np.array(buck.spechum)*units('kg/kg')

# calculate RH using metpy
buck['rh'] = mpcalc.relative_humidity_from_mixing_ratio(1013.25*units.hPa, 
                                                        temperature, 
                                                        mr).to('percent')

# calculate incoming LW radiation using Dilley and O'Brien empirical method
lw = lw_clr.dilleyobrien1998(buck.airtemp, buck.rh)
buck['LWRadAtm'] = lw
lw_mask = buck['LWRadAtm'] < 200
buck.loc[lw_mask, 'LWRadAtm'] = buck.LWRadAtm.mean()
buck['LWRadAtm'] = lw

# drop RH column
buck = buck.drop(columns=['rh'])

In [34]:
# Convert dataframe to xarray
dsx = buck.to_xarray()

# Loop through variables and add attributes from template forcing file
for data_var in dsx:
    dsx[data_var].attrs = template[data_var].attrs
    
# Add hru dimension
dsx = dsx.expand_dims(dim={'hru':1})

# Add gap-filled and datastep variables
dsx['gap_filled'] = xr.DataArray(np.ones((1,dsx.time.shape[0])),dims = ['hru','time'])
dsx['data_step'] = 3600 # 3600 seconds for 1hr timesteps

# Transpose gap filled variable to match dimensions with the rest
dsx['gap_filled'] = dsx['gap_filled'].T

# Convert all to float64
for var in dsx.data_vars:
    dsx[var] = dsx[var].astype(np.float64)
    
# Set hruID based on template
dsx['hruId'] = (xr.DataArray(np.ones((1))*template['hruId'].values,dims = ['hru'])).astype(np.int32)

In [ ]:
# copy attributes from template
count = 0
for var in dsx.data_vars:
    print(var,count)
    count += 1
    if count <= 7:
        attribs = dsx[var].attrs
        arr_t = dsx[var].values.T
        dsx[var] = xr.DataArray(dims = ['time','hru'],data = arr_t)
        dsx[var].attrs = attribs

In [37]:
# save to netcdf with proper time encoding
dsx.to_netcdf('../forcings/buck_era5_2016.nc',
                        encoding = {"time":
                                        {'dtype' : 'float64',
                                         'units' : 'hours since 1990-01-01 00:00:00',
                                         'calendar' : 'standard'}})